In [1]:
# -*- coding: utf-8 -*-
from __future__ import print_function
import os
import re

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine
from sklearn.metrics import euclidean_distances
from pyemd import emd

import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
import string

In [2]:
def check_value(w):
    w_string = str(w.encode('utf-8'))
    if(w_string in vocab_dict):
        return(vocab_dict[w_string])
        
    return 0

def lexicon_rate(lexicon, comment):
    vect = CountVectorizer(token_pattern=pattern, strip_accents=None).fit([lexicon, comment])
    v_1, v_2 = vect.transform([lexicon, comment])
    v_1 = v_1.toarray().ravel()
    v_2 = v_2.toarray().ravel()
    W_ = W[[check_value(w) for w in vect.get_feature_names()]]
    D_ = euclidean_distances(W_)
    v_1 = v_1.astype(np.double)
    v_2 = v_2.astype(np.double)
    v_1 /= v_1.sum()
    v_2 /= v_2.sum()
    D_ = D_.astype(np.double)
    D_ /= D_.max()
    lex=emd(v_1, v_2, D_)
    return(lex)

In [3]:
%%time
wv = KeyedVectors.load_word2vec_format('embedding/w2v-v1.wordvectors', unicode_errors="ignore")
wv.init_sims()

CPU times: user 1min 57s, sys: 3.08 s, total: 2min
Wall time: 2min


In [17]:
pattern = "(?u)\\b[\\w-]+\\b"

fp = np.memmap("data/embed.dat", dtype=np.double, mode='w+', shape=wv.vectors_norm.shape)
fp[:] = wv.vectors_norm[:]
with open("data/embed.vocab", "w") as f:
    for _, w in sorted((voc.index, word) for word, voc in wv.vocab.items()):
        print(w.encode('utf-8'), file=f)

vocab_len = len(wv.vocab)
del fp

W = np.memmap("data/embed.dat", dtype=np.double, mode="r", shape=(vocab_len, 300))

with open("data/embed.vocab") as f:
    vocab_list = map(str.strip, f.readlines())
vocab_dict={w:k for k, w in enumerate(vocab_list)}

In [35]:
# para cada fonte, percorrer a lista de arquivos em determianda pasta
folders = [
    'Indicadores/brasil_escola',
    'Indicadores/clue_words_pt',
    'Indicadores/acrobata_das_letras',
    'Indicadores/mundo_educacao',
    'Indicadores/Subjetivity_lexicon',
    'Indicadores/PersuativeEssays_UKP',
    'Indicadores/2020_Jonathan'
]


def get_indicators_by_sources(folders):
    indicators = dict()
    for folder in folders:
        argumentacao = list()
        for file_name in os.listdir(folder):    
            file_path = os.path.join(folder, file_name)
            with open(file_path) as f:
                argumentacao += [w.rstrip() for w in f.readlines()]
        argumentacao = ' '.join(argumentacao)
        indicators[folder.split('/')[1]] = argumentacao
    return indicators
    
def get_all_indicators():
    with open('Indicadores/preprocessed_indicators.txt') as f:
        argumentacao = [w.rstrip() for w in f.readlines()]
    return ' '.join(argumentacao)

argumentacoes = get_indicators_by_sources(folders)

In [96]:
argumentacao = argumentacoes[folders[6].split('/')[1]]

In [97]:
df = pd.read_csv("redações/redacoes_extraidas.csv")
df_not_arg = pd.read_csv("redações/Unlabeled/unlabeled.csv")
all_df = pd.concat([df, df_not_arg], ignore_index=True)
content = list(all_df.T.to_dict().values())

In [98]:
punctuation = string.punctuation \
        .replace('-', '') \
        .replace('_', '')
punctuation += "—«»"
punctuation = r"[{}]".format(punctuation)
re_trim = re.compile(r' +', re.UNICODE)

def loadStopWordsPT(filename):
    lines = [line.rstrip('\n').strip() for line in open(filename)]
    return lines

raw_stop_words = loadStopWordsPT('data/stop_words_preprocessed.txt')
        
def clean_stopwords(text, stop_words_list):
    list_words = text.split()
    list_clean_text = []
    for word in list_words:
        if word not in stop_words_list:
            list_clean_text.append(word)
    return " ".join(list_clean_text)

def clean(text):
    text = text.lower()
    text = re.sub(punctuation, " ", text)
    text = re_trim.sub(' ', text)

    return text

In [99]:
argumentacao = clean_stopwords(argumentacao, raw_stop_words)

for i in range(len(content)):
    content[i]['text'] = clean_stopwords(content[i]['text'], raw_stop_words)
    content[i]['text'] = clean(content[i]['text'])
    # content[i] = unicode(content[i], "utf-8")
    sent_text = nltk.sent_tokenize(content[i]['text'])
    content[i]['text'] = [sentence.encode("utf-8") for sentence in sent_text]

In [100]:
content[-1]

{'id': 697,
 'label': 'nao_argumento',
 'text': [b'aspectos devem considerados rela\xc3\xa7\xc3\xa3o isto legado hist\xc3\xb3rico-cultural desrespeito leis']}

In [101]:
def restore_text(list_sentences):
    original_text = ''
    for sentence in list_sentences:
        original_text += sentence.decode("utf-8")
    return original_text

In [102]:
def remove_dirty_sentences(list_sentences):
    clean_sentences = []
    for sentence in list_sentences:
        if(len(sentence.split()) > 2):
            clean_sentences.append(sentence)
    return clean_sentences

In [103]:
for i in range(len(content)):
    content[i]['text'] = remove_dirty_sentences(content[i]['text'])
content = [lista for lista in content if len(lista['text']) > 0]

In [104]:
lexicons_rates = list()
for sentences in content:
    arg_avg = 0

    for sentence in sentences['text']:
        arg_avg += lexicon_rate(argumentacao, sentence)
    arg_avg = arg_avg / float(len(sentences['text']))
    
    rates = list([sentences['id'], restore_text(sentences['text']),sentences['label'],arg_avg])
    lexicons_rates.append(rates)

In [105]:
df = pd.DataFrame(lexicons_rates, columns=['id','text','label','wmd_rate'])

In [106]:
df.to_csv(f"body/wmd_{folders[6].split('/')[1]}.csv", index=False)

In [79]:
# wv.wmdistance()
wv.wmdistance(['a'], ['a'])

0.0